Здесь происходит рисование разделённых по кластерам точек

In [5]:
import re
import gensim
import logging
import nltk.data
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from gensim.models import word2vec

from tqdm import tqdm
import numpy as np
import wget
import zipfile
import random

import sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from umap import UMAP

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [6]:
import numpy as np
words = []
xs = []
ys = []
zs = []
vectors = []
with open("words_coords_2.txt", encoding='utf8') as in_file:
#     line = in_file.readline()
    while True:
        line = in_file.readline()
        if line == '---\n':
            break
        else:
            res = line.split(' ')
            words.append(res[0])
            xs.append(float(res[2]))
            ys.append(float(res[4]))
            zs.append(float(res[5]))
            vectors.append(res[1:])
all_nouns = np.array(words)
for i in range(len(vectors)):
    for j in range(len(vectors[i])):
        vectors[i][j] = float(vectors[i][j])
vectors = np.array(vectors)

In [7]:
%store -r all_nouns

In [8]:
len(all_nouns)

59107

In [9]:
%store -r first_matrix

In [5]:
vectors = first_matrix

In [6]:
all_nouns[0]

'год_NOUN'

In [10]:
import pickle
import pandas as pd
import re
import numpy as np
from gensim.models import word2vec
from gensim.models import KeyedVectors
from tqdm import tqdm
from umap import UMAP

In [14]:
model = KeyedVectors.load_word2vec_format('araneum_upos_skipgram_300_2_2018.vec.gz')
#all_nouns = []
#for w in tqdm(model.index_to_key):
    #print(w)
#    if w.split('_')[-1] == 'NOUN':
#        all_nouns.append(w)

In [11]:
with open('araneum_all_nouns.txt', 'r', encoding='utf8') as f:
    all_nouns = f.read().strip('\n').split('\n')

In [12]:
def get_model_vectors(all_nouns):
    vectors_of_words = np.zeros((len(all_nouns), model.vector_size))
    for i, word in enumerate(all_nouns):
        try:
            vectors_of_words[i] = model[word]
        except:
            print(word)
    return vectors_of_words

In [15]:
vectors = get_model_vectors(all_nouns)

In [16]:
def get_matrix(all_nouns, vectors):
    vectors_of_words = np.zeros((len(all_nouns), len(vectors[0])))
    for i, word in enumerate(all_nouns):
        try:
            vectors_of_words[i] = vectors[i]
        except:
            print(word)
    return vectors_of_words

def lsa_matrix(vectors_of_words, n_components, n_iter=100):
   #lsa_obj = TruncatedSVD(n_components=n_components, n_iter=n_iter, random_state=42)
    lsa_data = UMAP().fit_transform(vectors_of_words)
    return lsa_data

In [17]:
def sort_results(lsa_data, all_nouns):    
    sorted_scores_indx = np.argsort(lsa_data, axis=0)[::-1]
    result = np.array(all_nouns)[sorted_scores_indx.ravel()]
    result_nums = np.array(lsa_data)[sorted_scores_indx.ravel()]
    return result, result_nums

def sort_results2(lsa_data, all_nouns, all_lsa):    
    sorted_scores_indx = np.argsort(lsa_data, axis=0)[::-1]
    result = np.array(all_nouns)[sorted_scores_indx.ravel()]
    result_nums = np.array(all_lsa)[sorted_scores_indx.ravel()]
    return result, result_nums

In [18]:
def get_n_iterations(all_nouns, iterations):
    dict_iters = {'0': [all_nouns]}
    for i in range(iterations):
        iter_name = str(i + 1)
        dict_iters[iter_name] = []
        for el in dict_iters[str(i)]:
            if len(el) > 50:
                first_matrix = get_matrix(el, vectors)
                first_lsa = lsa_matrix(first_matrix, i + 1, 200)
                first_result, first_result_num = sort_results([v[0] for v in first_lsa], el)
                part_of_list = len(first_result) // 5
                dict_iters[iter_name].append(first_result[:part_of_list])
                dict_iters[iter_name].append(first_result[2*part_of_list:])
            else:
                dict_iters[iter_name].append(el)
                
    return dict_iters

In [19]:
from sklearn.cluster import KMeans
def clustering(vectors_of_words,  eps=3, min_samples=1000):
    #clustering_data = DBSCAN(eps=eps, min_samples=first_matrix.shape[0]/5).fit(vectors_of_words)
    clustering_data = sklearn.cluster.KMeans(n_clusters=vectors_of_words.shape[0]//eps).fit_predict(vectors_of_words)
    return clustering_data

In [20]:
def separate_words(first_matrix, first_lsa, all_nouns, clust_size):
    first_clustering = clustering(first_matrix, clust_size)
    m, d = np.mean(first_lsa), np.std(first_lsa)

    processed = [-1 for i in range(max(first_clustering)+1)]
    distr = [[0, 0, 0] for i in range(max(first_clustering)+1)]

    for val, cl in zip(first_lsa, first_clustering):
        if val < m - d:
            distr[cl][0] += 1
        elif val > m + d:
            distr[cl][2] += 1
        else:
            distr[cl][1] += 1

    words = [[], [], []]
    for word, cl in zip(all_nouns, first_clustering):
        pos = np.argmax(distr[cl])
        words[pos].append(word)

    return words

In [21]:
def separate_words2(first_matrix, first_lsa, all_nouns, clust_size):
    #first_clustering = clustering(first_matrix, clust_size)
    m, d = np.mean(first_lsa), np.std(first_lsa)

#     processed = [-1 for i in range(max(first_clustering)+1)]
#     distr = [[0, 0, 0] for i in range(max(first_clustering)+1)]

    words = [[], [], []]
    for word, val in zip(all_nouns, first_lsa):
        if val < m - 1/2*d:
            words[0].append(word)
        elif val > m + 1/2*d:
            words[2].append(word)
        else:
            words[1].append(word)

    return words

In [22]:
def get_n_iterations2(all_nouns, iterations):
    
    clust_size = {'0': 100, '1': 30, '2': 30, '3': 10, '4':10, '5':10, '6':10, '7':10, '8':10, '9':10, '10':10}
    dict_iters = {'0': [all_nouns]}
    dict_iters_num = {'0': []}
    for i in range(iterations):
        print(i)
        iter_name = str(i + 1)
        dict_iters[iter_name] = []
        dict_iters_num[iter_name] = []
        for one_nouns_list in dict_iters[str(i)]:
            if len(one_nouns_list) > 20:
                first_matrix = get_matrix(one_nouns_list, vectors)
                first_lsa = lsa_matrix(first_matrix, i + 2, 200)
                words = separate_words2(first_matrix, [i[-1] for i in first_lsa], one_nouns_list, clust_size[str(i)])
                dict_iters[iter_name].extend(words)
            else:
                dict_iters[iter_name].append(one_nouns_list)
            
            
            #first_result, first_result_num = sort_results2([v[0] for v in first_lsa], el, first_lsa)
            #half_of_list = len(first_result) // 3
            #dict_iters[iter_name].append(first_result[:half_of_list])
            #dict_iters[iter_name].append(first_result[2*half_of_list:])
            #dict_iters_num[iter_name].append(first_result_num[:half_of_list])
            #dict_iters_num[iter_name].append(first_result_num[2*half_of_list:])
            
    return dict_iters #, dict_iters_num

In [23]:
%%time
res = get_n_iterations2(all_nouns, 6)

0
1
2
3
4
5
Wall time: 22min 36s


In [24]:
import pickle
with open('dict_umap.pkl', 'wb') as f:
    pickle.dump(res, f)

In [25]:
import pickle
with open('dict_umap.pkl', 'rb') as f:
    res = pickle.load(f)

In [26]:
len([len(i) for i in res['4']])

81

In [27]:
max([len(i) for i in res['4']])

716

In [40]:
res['1'][1][:100]

['год_NOUN',
 'время_NOUN',
 'работа_NOUN',
 'день_NOUN',
 'компания_NOUN',
 'система_NOUN',
 'вид_NOUN',
 'сайт_NOUN',
 'страна_NOUN',
 'мир_NOUN',
 'часть_NOUN',
 'программа_NOUN',
 'проект_NOUN',
 'центр_NOUN',
 'информация_NOUN',
 'уровень_NOUN',
 'сторона_NOUN',
 'цена_NOUN',
 'процесс_NOUN',
 'количество_NOUN',
 'услуга_NOUN',
 'форма_NOUN',
 'управление_NOUN',
 'рынок_NOUN',
 'число_NOUN',
 'интернет_NOUN',
 'использование_NOUN',
 'игра_NOUN',
 'школа_NOUN',
 'момент_NOUN',
 'деньги_NOUN',
 'месяц_NOUN',
 'производство_NOUN',
 'клиент_NOUN',
 'начало_NOUN',
 'правило_NOUN',
 'конец_NOUN',
 'документ_NOUN',
 'образование_NOUN',
 'час_NOUN',
 'магазин_NOUN',
 'данные_NOUN',
 'модель_NOUN',
 'бизнес_NOUN',
 'технология_NOUN',
 'создание_NOUN',
 'срок_NOUN',
 'стоимость_NOUN',
 'выбор_NOUN',
 'способ_NOUN',
 'предприятие_NOUN',
 'счет_NOUN',
 'вариант_NOUN',
 'рубль_NOUN',
 'тип_NOUN',
 'сеть_NOUN',
 'период_NOUN',
 'оборудование_NOUN',
 'проведение_NOUN',
 'имя_NOUN',
 'изменение_N

In [31]:
len(res['0'][0])

46074

In [15]:
#res['3'][26]

In [14]:
len(res['3'][:3])

3

In [17]:
#res['3']

In [ ]:
res['3']

In [55]:
import pickle
with open('dict_with_distance.pkl', 'wb') as f:
    pickle.dump(res, f)

In [13]:
import pickle
with open('saved_dictionary.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [14]:
len(loaded_dict['3'][4])

1319

## Рисуем

Тут мы рисуем каждый кластер в 2D с подписями к точкам

In [31]:
matrix_of_all_2D = get_matrix(all_nouns, vectors)
all_coords_2D = lsa_matrix(matrix_of_all_2D, 3, 200)

In [36]:
import matplotlib.pyplot as plt
import numpy as np
import random

def get_one_graph(list_of_clusters, all_nouns, all_coords_2D, name='test'):
     
    fig = plt.figure(dpi=600, figsize = (25, 15))
    
    colors = ['#4B0082', '#FFA500', '#9ACD32']
    
    for i, cluster in enumerate(list_of_clusters):
        
        x, y = [], []
        
        for word in cluster:
            point_coords = all_coords_2D[list(all_nouns).index(word)]
            x.append(point_coords[0])
            y.append(point_coords[1])
            plt.annotate(word, xy=(point_coords[0], point_coords[1]))

        plt.scatter(x, y, color=colors[i])

    fig.savefig(name)

In [37]:
get_one_graph(res['3'][3:6], all_nouns, all_coords_2D, name='teeeeeest')

## Рисуем в 3D

In [49]:
matrix_of_all = get_matrix(all_nouns, vectors)
coords = lsa_matrix(matrix_of_all, 4, 200)

In [50]:
len(all_nouns) == len(set(all_nouns))

True

In [51]:
list(all_nouns).index('синклиналь')

ValueError: 'синклиналь' is not in list

In [57]:
import plotly.express as px

actions = [1/27*(i+1) for i in range(len(res['3']))]

x, y, z, c, t, s = [], [], [], [], [], []

for i, layer in enumerate([res['3']]):
    for a, cluster in enumerate(layer):
        for word in cluster:
            point_coords = coords[list(all_nouns).index(word)]

            x.append(point_coords[0])
            y.append(point_coords[1])
            z.append(point_coords[2])
            s.append(point_coords[3])
            c.append(actions[a])
            t.append(word.lower())
        

fig = px.scatter_3d(x=x, y=y, z=z,
              color=c, text=t, size=[p - min(s) for p in s])#, size=c)
fig.show()

In [56]:
point_coords

array([ 1.19592324e+03, -1.30060726e+01,  4.33915000e+00, -1.62190108e-01])

In [23]:
loaded_dict['3'][1]

['камыш',
 'хомут',
 'мельница',
 'барабан',
 'игла',
 'колесо',
 'рубашка',
 'бур',
 'лес',
 'косяк',
 'молоток',
 'ножницы',
 'шкура',
 'тележка',
 'коса',
 'кустарник',
 'котёл',
 'нож',
 'растительность',
 'растение',
 'колодка',
 'животное',
 'трава',
 'гриб',
 'бревно',
 'кузов',
 'башмак',
 'луг',
 'подставка',
 'нитка',
 'лопатка',
 'конёк',
 'орудие',
 'плот',
 'птица',
 'тур',
 'шуба',
 'мох',
 'гвоздь',
 'носок',
 'ковш',
 'бельё',
 'качели',
 'лопата',
 'стебель',
 'прут',
 'войлок',
 'короб',
 'ремень',
 'алт',
 'вилы',
 'серп',
 'плеть',
 'кос',
 'кет',
 'вилка',
 'тат',
 'произрастание',
 'плотник',
 'обувь',
 'инвентарь',
 'крюк',
 'тростник',
 'крючок',
 'мех',
 'колотушка',
 'горн',
 'шерсть',
 'шашка',
 'стук',
 'пила',
 'обруч',
 'люлька',
 'телеги',
 'рукоятка',
 'сарафан',
 'дудка',
 'наковальня',
 'бубен',
 'верёвка',
 'клубок',
 'телега',
 'платок',
 'рубаха',
 'ружьё',
 'трость',
 'рожок',
 'шест',
 'кузнец',
 'палочка',
 'берёста',
 'повозка',
 'заросль',
 'аз

In [39]:
fig.show()

In [29]:
fig.write_html("test.html")